Start with the standard imports we have used for every notebook in this class.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

#These are one time load of SCHOOL data steps.  
#START SECTION OF DO NOT RUN THESE AGAIN

Each of the datasheets downloaded from ELSI had download metadata on the top of them and total and key information on the bottom of them that were not data rows.  This metadata, total, and key information was manually deleted before import.  Some of the files had ="0" instead of 0 in the cells.  This was found/replaced before import using the sed -i '' 's/="0"/0/g' *.csv command from the terminal.

In [46]:
#CITATION: This is the data from National Center for Education Statistics on Schools
#Schools for all 50 states and Washington, D.C.
#http://nces.ed.gov/ccd/elsi/
#ELSI Root Data Source: U.S. Department of Education National Center for Education Statistics Common Core of Data (CCD) "Public Elementary/Secondary School Universe Survey" 2012-13 v.1a  2013-14 v.1a.
#KEY:
#† indicates that the data are not applicable.
#– indicates that the data are missing.
#‡ indicates that the data do not meet NCES data quality standards.

schoolinformation = pd.read_csv("tempdata/2013-2014 SCHOOL Information Tab.csv", dtype=np.str)
schoolcharacteristicsa = pd.read_csv("tempdata/2013-2014 SCHOOLS CharacteristicsA Tab.csv", dtype=np.str)
schoolcharacteristicsb = pd.read_csv("tempdata/2013-2014 SCHOOLS CharacteristicsB Tab.csv", dtype=np.str)
schoolenrollment = pd.read_csv("tempdata/2013-2014 SCHOOL Enrollments Tab.csv", dtype=np.str)
schoolenrollmentdetails = pd.read_csv("tempdata/2013-2014 SCHOOL Enrollment Details Tab.csv", dtype=np.str)
schoolenrollmentK3 = pd.read_csv("tempdata/2013-2014 SCHOOL Enrollment by Grade, Race-Ethnicity, and Gender Tab PreK-3.csv", dtype=np.str)
schoolenrollment48 = pd.read_csv("tempdata/2013-2014 SCHOOL Enrollment by Grade, Race-Ethnicity and Gender Tab 4-8.csv", dtype=np.str)
schoolenrollment912 = pd.read_csv("tempdata/2013-2014 SCHOOL Enrollment by Grade, Race-Ethnicity and Gender Tab 9-Ungraded.csv", dtype=np.str)
schoolteacherstaff = pd.read_csv("tempdata/2013-2014 SCHOOL Teacher and Staff Tab.csv", dtype=np.str)

Check the lengths of the datasets to see if we have a row for every school.

In [47]:
print len(schoolinformation)
print len(schoolcharacteristicsa) #This is the only dataset that doesn't have a row for every school
print len(schoolcharacteristicsb)
print len(schoolenrollment)
print len(schoolenrollmentdetails)
print len(schoolenrollmentK3)
print len(schoolenrollment48)
print len(schoolenrollment912)
print len(schoolteacherstaff)

99074
79897
99074
99074
99074
99074
99074
99074
99074


Drop all of the duplicate columns.

In [48]:
schoolcharacteristicsb = schoolcharacteristicsb.drop(schoolcharacteristicsb.columns[[0, 1, 2]], 1)
schoolenrollment = schoolenrollment.drop(schoolenrollment.columns[[0, 1]], 1)
schoolenrollmentdetails = schoolenrollmentdetails.drop(schoolenrollmentdetails.columns[[0, 1]], 1)
schoolenrollmentK3 = schoolenrollmentK3.drop(schoolenrollmentK3.columns[[0, 1]], 1)
schoolenrollment48 = schoolenrollment48.drop(schoolenrollment48.columns[[0, 1]], 1)
schoolenrollment912 = schoolenrollment912.drop(schoolenrollment912.columns[[0, 1, 72]], 1)
schoolteacherstaff = schoolteacherstaff.drop(schoolteacherstaff.columns[[0, 1, 4]], 1)

In [49]:
schoolinformation.head()

,﻿School Name,State Name [Public School] Latest available year,State Name [Public School] 2013-14,State Abbr [Public School] Latest available year,School Name [Public School] 2013-14,School ID - NCES Assigned [Public School] Latest available year,Agency Name [Public School] 2013-14,Agency ID - NCES Assigned [Public School] Latest available year,County Name [Public School] 2013-14,County Number [Public School] 2013-14,ANSI/FIPS State Code [Public School] Latest available year,Years School Reported Data [Public School] Latest available year,Years School Did Not Report Data [Public School] Latest available year,Location Address [Public School] 2013-14,Location City [Public School] 2013-14,Location State Abbr [Public School] 2013-14,Location ZIP [Public School] 2013-14,Location ZIP4 [Public School] 2013-14,Mailing Address [Public School] 2013-14,Mailing City [Public School] 2013-14,Mailing State Abbr [Public School] 2013-14,Mailing ZIP [Public School] 2013-14,Mailing ZIP4 [Public School] 2013-14,Phone Number [Public School] 2013-14
0,"EDWARD ""DUKE"" ELLINGTON @ BECKHAM",Michigan,Michigan,MI,"EDWARD ""DUKE"" ELLINGTON @ BECKHAM",261200004838,DETROIT CITY SCHOOL DISTRICT,2612000,WAYNE COUNTY,26163,26,1986-2013,NaN,9860 PARK DR,DETROIT,MI,48213,7102,9860 PARK DRIVE,DETROIT,MI,48213,7102,3138528500
1,"J.M. ""CHICK"" BUCKBEE JUVENILE CENTER",West Virginia,West Virginia,WV,"J.M. ""CHICK"" BUCKBEE JUVENILE CENTER",540051101159,INSTITUTIONAL EDUCATIONAL PROGRAMS,5400511,HAMPSHIRE COUNTY,54027,54,2003-2013,1986-2002,1 JERRY LN,AUGUSTA,WV,26704,†,#1 JERRY LANE,AUGUSTA,WV,26704,†,3044961369
2,"JAMES H. ""TIGER"" MORTON JUVENILE CENTER",West Virginia,West Virginia,WV,"JAMES H. ""TIGER"" MORTON JUVENILE CENTER",540051101146,INSTITUTIONAL EDUCATIONAL PROGRAMS,5400511,KANAWHA COUNTY,54039,54,2003-2013,1986-2002,60 MANFRED HOLLAND WAY,DUNBAR,WV,25064,†,60 MANFRED HOLLAND WAY,DUNBAR,WV,25064,†,3047663502
3,"MARY HARRIS ""MOTHER"" JONES ELEMENTARY",Maryland,Maryland,MD,"MARY HARRIS ""MOTHER"" JONES ELEMENTARY",240051001257,PRINCE GEORGE'S COUNTY PUBLIC SCHOOLS,2400510,PRINCE GEORGE'S COUNTY,24033,24,2002-2013,1986-2001,2405 TECUMSEH ST,ADELPHI,MD,20783,1658,2405 TECUMSEH ST,ADELPHI,MD,20783,1658,3014087900
4,"PEMAYETV EMAHAKV CHARTER ""OUR WAY SCHOOL""",Florida,Florida,FL,"PEMAYETV EMAHAKV CHARTER ""OUR WAY SCHOOL""",120066004849,GLADES,1200660,GLADES COUNTY,12043,12,2006-2013,1986-2005,100 EAST HARNEY POND RD NE,OKEECHOBEE,FL,34974,2867,100 E HARNEY POND RD NE,OKEECHOBEE,FL,34974,2867,8634672501


Join all of the school datasets.  The dataset schoolcharacteristicsa needs special treatment, as it is the only dataset that is missing rows for the schools.  All of the other datasets can be joined by ID without issue.

In [50]:
joinedschool = schoolinformation.merge(schoolcharacteristicsa, 'outer', 'School ID - NCES Assigned [Public School] Latest available year', suffixes=('', '_DEL'))
joinedschool = joinedschool.drop(joinedschool[[24, 25]], 1)
joinedschool = joinedschool.join([schoolcharacteristicsb, schoolenrollment, schoolenrollmentdetails, schoolenrollmentK3, schoolenrollment48, schoolenrollment912, schoolteacherstaff])
#Need to get rid of . in column names to store them to mongo
joinedschool = joinedschool.rename(columns=lambda x: x.replace('.', ''))
#Need to get rid of Excel syntax ="" from some of the columns
joinedschool['School ID - NCES Assigned [Public School] Latest available year'] = joinedschool['School ID - NCES Assigned [Public School] Latest available year'].map(lambda x: x.lstrip('="').rstrip('"'))
joinedschool['Agency ID - NCES Assigned [Public School] Latest available year'] = joinedschool['Agency ID - NCES Assigned [Public School] Latest available year'].map(lambda x: x.lstrip('="').rstrip('"'))
joinedschool['County Number [Public School] 2013-14'] = joinedschool['County Number [Public School] 2013-14'].map(lambda x: x.lstrip('="').rstrip('"'))
joinedschool['ANSI/FIPS State Code [Public School] Latest available year'] = joinedschool['ANSI/FIPS State Code [Public School] Latest available year'].map(lambda x: x.lstrip('="').rstrip('"'))
joinedschool['Location ZIP [Public School] 2013-14'] = joinedschool['Location ZIP [Public School] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joinedschool['Location ZIP4 [Public School] 2013-14'] = joinedschool['Location ZIP4 [Public School] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joinedschool['Mailing ZIP [Public School] 2013-14'] = joinedschool['Mailing ZIP [Public School] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joinedschool['Mailing ZIP4 [Public School] 2013-14'] = joinedschool['Mailing ZIP4 [Public School] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
#If by chance any rows have NaN, replace with the ELSI standard for missing data '–'
joinedschool = joinedschool.fillna('–')

joinedschool.head()

,﻿School Name,State Name [Public School] Latest available year,State Name [Public School] 2013-14,State Abbr [Public School] Latest available year,School Name [Public School] 2013-14,School ID - NCES Assigned [Public School] Latest available year,Agency Name [Public School] 2013-14,Agency ID - NCES Assigned [Public School] Latest available year,County Name [Public School] 2013-14,County Number [Public School] 2013-14,ANSI/FIPS State Code [Public School] Latest available year,Years School Reported Data [Public School] Latest available year,Years School Did Not Report Data [Public School] Latest available year,Location Address [Public School] 2013-14,Location City [Public School] 2013-14,Location State Abbr [Public School] 2013-14,Location ZIP [Public School] 2013-14,Location ZIP4 [Public School] 2013-14,Mailing Address [Public School] 2013-14,Mailing City [Public School] 2013-14,Mailing State Abbr [Public School] 2013-14,Mailing ZIP [Public School] 2013-14,Mailing ZIP4 [Public School] 2013-14,Phone Number [Public School] 2013-14,School Type [Public School] 2013-14,Agency Type [District] 2013-14,Operational Status [Public School] 2013-14,Charter School [Public School] 2013-14,Magnet School [Public School] 2013-14,Shared Time School [Public School] 2013-14,Urban-centric Locale [Public School] 2013-14,School-wide Title I [Public School] 2013-14,Title I Eligible School [Public School] 2013-14,Title I School Status [Public School] 2013-14,CBSA Name [District] 2013-14,CBSA ID [District] 2013-14,CSA Name [District] 2013-14,CSA ID [District] 2013-14,Latitude [Public School] 2013-14,Longitude [Public School] 2013-14,State School ID [Public School] 2013-14,State Agency ID [Public School] 2013-14,Congressional Code [Public School] 2013-14,Reconstituted flag [Public School] 2013-14,Reconstituted year [Public School] 2013-14,Virtual School Status [Public School] 2013-14,National School Lunch Program [Public School] 2013-14,Charter Authorizer Primary [Public School] 2013-14,Charter Authorizer Secondary [Public School] 2013-14,Lowest Grade Offered [Public School] 2013-14,...,Grade 10 Students - White - male [Public School] 2013-14,Grade 10 Students - White - female [Public School] 2013-14,Grade 10 Students - Hawaiian Nat/Pacific Isl - male [Public School] 2013-14,Grade 10 Students - Hawaiian Nat/Pacific Isl - female [Public School] 2013-14,Grade 10 Students - Two or More Races - male [Public School] 2013-14,Grade 10 Students - Two or More Races - female [Public School] 2013-14,Grade 11 Students - American Indian/Alaska Native - male [Public School] 2013-14,Grade 11 Students - American Indian/Alaska Native - female [Public School] 2013-14,Grade 11 Students - Asian or Asian/Pacific Islander - male [Public School] 2013-14,Grade 11 Students - Asian or Asian/Pacific Islander - female [Public School] 2013-14,Grade 11 Students - Hispanic - male [Public School] 2013-14,Grade 11 Students - Hispanic - female [Public School] 2013-14,Grade 11 Students - Black - male [Public School] 2013-14,Grade 11 Students - Black - female [Public School] 2013-14,Grade 11 Students - White - male [Public School] 2013-14,Grade 11 Students - White - female [Public School] 2013-14,Grade 11 Students - Hawaiian Nat/Pacific Isl - male [Public School] 2013-14,Grade 11 Students - Hawaiian Nat/Pacific Isl - female [Public School] 2013-14,Grade 11 Students - Two or More Races - male [Public School] 2013-14,Grade 11 Students - Two or More Races - female [Public School] 2013-14,Grade 12 Students - American Indian/Alaska Native - male [Public School] 2013-14,Grade 12 Students - American Indian/Alaska Native - female [Public School] 2013-14,Grade 12 Students - Asian or Asian/Pacific Islander - male [Public School] 2013-14,Grade 12 Students - Asian or Asian/Pacific Islander - female [Public School] 2013-14,Grade 12 Students - Hispanic - male [Public School] 2013-14,Grade 12 Students - Hispanic - female [Public School] 2013-14,Grade 12 Students - Black - male [Public School] 2013-14,Gra

In [52]:
joinedschool.to_csv("tempdata/schools.csv")

#END SECTION OF DO NOT RUN THESE AGAIN

#START Exploratory Data Analysis

Load the schools data.

In [53]:
schoolsdf = pd.read_csv("tempdata/schools.csv")
schoolsdf.head()

,Unnamed: 0,﻿School Name,State Name [Public School] Latest available year,State Name [Public School] 2013-14,State Abbr [Public School] Latest available year,School Name [Public School] 2013-14,School ID - NCES Assigned [Public School] Latest available year,Agency Name [Public School] 2013-14,Agency ID - NCES Assigned [Public School] Latest available year,County Name [Public School] 2013-14,County Number [Public School] 2013-14,ANSI/FIPS State Code [Public School] Latest available year,Years School Reported Data [Public School] Latest available year,Years School Did Not Report Data [Public School] Latest available year,Location Address [Public School] 2013-14,Location City [Public School] 2013-14,Location State Abbr [Public School] 2013-14,Location ZIP [Public School] 2013-14,Location ZIP4 [Public School] 2013-14,Mailing Address [Public School] 2013-14,Mailing City [Public School] 2013-14,Mailing State Abbr [Public School] 2013-14,Mailing ZIP [Public School] 2013-14,Mailing ZIP4 [Public School] 2013-14,Phone Number [Public School] 2013-14,School Type [Public School] 2013-14,Agency Type [District] 2013-14,Operational Status [Public School] 2013-14,Charter School [Public School] 2013-14,Magnet School [Public School] 2013-14,Shared Time School [Public School] 2013-14,Urban-centric Locale [Public School] 2013-14,School-wide Title I [Public School] 2013-14,Title I Eligible School [Public School] 2013-14,Title I School Status [Public School] 2013-14,CBSA Name [District] 2013-14,CBSA ID [District] 2013-14,CSA Name [District] 2013-14,CSA ID [District] 2013-14,Latitude [Public School] 2013-14,Longitude [Public School] 2013-14,State School ID [Public School] 2013-14,State Agency ID [Public School] 2013-14,Congressional Code [Public School] 2013-14,Reconstituted flag [Public School] 2013-14,Reconstituted year [Public School] 2013-14,Virtual School Status [Public School] 2013-14,National School Lunch Program [Public School] 2013-14,Charter Authorizer Primary [Public School] 2013-14,Charter Authorizer Secondary [Public School] 2013-14,...,Grade 10 Students - White - male [Public School] 2013-14,Grade 10 Students - White - female [Public School] 2013-14,Grade 10 Students - Hawaiian Nat/Pacific Isl - male [Public School] 2013-14,Grade 10 Students - Hawaiian Nat/Pacific Isl - female [Public School] 2013-14,Grade 10 Students - Two or More Races - male [Public School] 2013-14,Grade 10 Students - Two or More Races - female [Public School] 2013-14,Grade 11 Students - American Indian/Alaska Native - male [Public School] 2013-14,Grade 11 Students - American Indian/Alaska Native - female [Public School] 2013-14,Grade 11 Students - Asian or Asian/Pacific Islander - male [Public School] 2013-14,Grade 11 Students - Asian or Asian/Pacific Islander - female [Public School] 2013-14,Grade 11 Students - Hispanic - male [Public School] 2013-14,Grade 11 Students - Hispanic - female [Public School] 2013-14,Grade 11 Students - Black - male [Public School] 2013-14,Grade 11 Students - Black - female [Public School] 2013-14,Grade 11 Students - White - male [Public School] 2013-14,Grade 11 Students - White - female [Public School] 2013-14,Grade 11 Students - Hawaiian Nat/Pacific Isl - male [Public School] 2013-14,Grade 11 Students - Hawaiian Nat/Pacific Isl - female [Public School] 2013-14,Grade 11 Students - Two or More Races - male [Public School] 2013-14,Grade 11 Students - Two or More Races - female [Public School] 2013-14,Grade 12 Students - American Indian/Alaska Native - male [Public School] 2013-14,Grade 12 Students - American Indian/Alaska Native - female [Public School] 2013-14,Grade 12 Students - Asian or Asian/Pacific Islander - male [Public School] 2013-14,Grade 12 Students - Asian or Asian/Pacific Islander - female [Public School] 2013-14,Grade 12 Students - Hispanic - male [Public School] 2013-14,Grade 12 Students - Hispanic - female [Public School] 2013-14,Grade 12 Students - Black - male [Public School] 2013-14,Grade 12 Students - Black - female [P

This is the full list of all column names in the schools dataframe.

In [54]:
for col in schoolsdf.columns:
    print col

Unnamed: 0
﻿School Name
State Name [Public School] Latest available year
State Name [Public School] 2013-14
State Abbr [Public School] Latest available year
School Name [Public School] 2013-14
School ID - NCES Assigned [Public School] Latest available year
Agency Name [Public School] 2013-14
Agency ID - NCES Assigned [Public School] Latest available year
County Name [Public School] 2013-14
County Number [Public School] 2013-14
ANSI/FIPS State Code [Public School] Latest available year
Years School Reported Data [Public School] Latest available year
Years School Did Not Report Data [Public School] Latest available year
Location Address [Public School] 2013-14
Location City [Public School] 2013-14
Location State Abbr [Public School] 2013-14
Location ZIP [Public School] 2013-14
Location ZIP4 [Public School] 2013-14
Mailing Address [Public School] 2013-14
Mailing City [Public School] 2013-14
Mailing State Abbr [Public School] 2013-14
Mailing ZIP [Public School] 2013-14
Mailing ZIP4 [Public

In [55]:
#NOTE: The first categorical column starts at School Type.
#NOTE: Skip over any unique information like school name, address, ID, etc.
#TODO: Run this for every column, and decide what group the column needs to go into using the results.
#NOTE: REMEMBER, high counts of – and ‡ in the below might make the column unusable.  Do not worry as much about † (N/A) because it's probably that the school is an elementary school and the column is applicable for high schools or vice versa. 
#KEY:
#† indicates that the data are not applicable.
#– indicates that the data are missing.
#‡ indicates that the data do not meet NCES data quality standards.
schoolsdf['School Type [Public School] 2013-14'].value_counts()

1-Regular school              72586
–                             19177
4-Alternative/other school     4344
2-Special education school     1743
3-Vocational school            1224
dtype: int64

Think like homework 3, and we'll need to do some separation of types of columns.  Below is just to give an example - I would see us expanding this out.  If we find a data column that we shouldn't use because the response rate is super low, let's put that in invalid columns.  Percent variables go into PERCENT_VARS, and columns amenable to standardization go into STANDARDIZABLE.

In [ ]:
#TODO: Using the descriptive results of value_counts() in the earlier steps, setup our lists of types of columns.
INVALID_COLUMNS=[]
PERCENT_VARS=[]
STANDARDIZABLE = PERCENT_VARS + []

In [11]:
#NOTE: This does the same as value_counts() above, but lists for all columns.  I find this more difficult to work with, so I moved off of this in favor for what I did above.  I'm leaving this here in case someone else finds this way easier.
for col in schoolsdf:
    print schoolsdf[col].value_counts()

2047     1
96890    1
2708     1
661      1
6806     1
4759     1
27288    1
25241    1
31386    1
29339    1
19100    1
17053    1
23198    1
21151    1
98981    1
76464    1
74417    1
80562    1
78515    1
68276    1
66229    1
72374    1
70327    1
12947    1
14994    1
8849     1
33413    1
84604    1
82557    1
88702    1
        ..
58742    1
60791    1
38264    1
40313    1
34170    1
25958    1
30052    1
42302    1
70984    1
44351    1
87360    1
89409    1
83266    1
85315    1
95556    1
97605    1
91462    1
93511    1
73033    1
19811    1
66890    1
68939    1
79180    1
81229    1
75086    1
77135    1
21856    1
23905    1
17762    1
0        1
dtype: int64
LINCOLN ELEMENTARY SCHOOL                       89
LINCOLN ELEMENTARY                              73
WASHINGTON ELEMENTARY SCHOOL                    70
JEFFERSON ELEMENTARY                            67
CENTRAL ELEMENTARY SCHOOL                       64
WASHINGTON ELEMENTARY                           55
JEFFERSON 